## todo: referrer

In [ ]:
import requests, re, time
from bs4 import BeautifulSoup
from urllib.parse import urlparse


In [ ]:
def _eval_resp(resp:requests.Response):
    headers = resp.headers
    pattern = r"^([a-z]+)/([a-z]+)"
    try:
        m = re.match(pattern,headers['Content-Type'])
    except KeyError:
        return None, None
    typ = m.group(1)
    filetype = m.group(2)
    return typ, filetype


In [ ]:
def extract_all_links(page_source):
    # Regex zum Extrahieren von href-Werten
    href_pattern = r'href=["\'](.*?)["\']'
    # Alle Übereinstimmungen finden
    hrefs = re.findall(href_pattern, page_source)
    return hrefs
def gehört_url_zu_hauptdomain(url, hauptdomain):
    parsed_url = urlparse(url)
    # Überprüfe, ob die Hauptdomain am Ende des Hostnamens steht
    netloc = parsed_url.netloc
    parts = netloc.split(".")
    tl_domain = parts[-1]
    sl_domain = parts[-2]
    return hauptdomain == sl_domain

In [ ]:
def list_all_public_links(base_url:str)->dict[str, list[str]]:
    root_domain = urlparse(base_url).netloc.split(".")[-2]
    res_d = {"text/html": [base_url]}
    visited_set = {base_url}
    referrer_dict = {base_url: None}
    links = extract_all_links(requests.get(base_url).text)
    queue_set = set(links)
    for link in links:
        referrer_dict[link] = [base_url]
    while queue_set:
        current_elem = queue_set.pop()
        if current_elem.startswith("mailto:"):
            continue
        if not current_elem.startswith("https://"):
            if current_elem.startswith("/"):
                current_elem = base_url+current_elem
            else:
                current_elem = base_url+"/"+current_elem

        pattern = r'^(https?:\/\/.*?)(?=(https?:\/\/|$))'
        current_elem = re.match(pattern, current_elem).group(1)
        
        time.sleep(5)
        print(current_elem)
        print(f"Besuche: {current_elem}")
        try:
            resp = requests.get(current_elem)
        except Exception as e:
            print(f"Fehler beim Abrufen von {current_elem}: {e}")
            continue

        visited_set.add(current_elem)
        keys = res_d.keys()
        tf = _eval_resp(resp)
        content = "/".join(tf)
        if content not in keys:
            res_d[content] = []
        res_d[content].append(current_elem)
        if tf != ("text", "html"):
            continue
        if not gehört_url_zu_hauptdomain(current_elem, root_domain):
            continue
        new_links = set(extract_all_links(resp.text))
        links_not_visited = new_links - visited_set
        for link in links_not_visited:
            if link in referrer_dict:
                referrer_dict[link].append(current_elem)
            else:
                referrer_dict[link] = [current_elem]

        queue_set.update(links_not_visited)
    return res_d, referrer_dict

In [13]:
url = "https://www.athexgroup.gr/web/guest/companies-information-corporate-actions/-/asset_publisher/E8Xk2zINhmZA/document/id/7590823?controlPanelCategory=portlet_101_INSTANCE_E8Xk2zINhmZA&redirect=https%3A%2F%2Fwww.athexgroup.gr%2Fweb%2Fguest%2Fcompanies-information-corporate-actions%3Fp_p_id%3D101_INSTANCE_E8Xk2zINhmZA%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26controlPanelCategory%3Dportlet_101_INSTANCE_E8Xk2zINhmZA%26_101_INSTANCE_E8Xk2zINhmZA_"

In [14]:
resp = requests.get(url)

In [21]:
print(resp.headers)

{'Date': 'Mon, 11 Nov 2024 14:31:26 GMT', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=63072000; includeSubdomains;', 'X-Frame-Options': 'SAMEORIGIN', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Content-Type': 'text/html;charset=UTF-8', 'Set-Cookie': 'JSESSIONID=E934EBB7C0C00B0FCFC4C2DB4ECE8365.jvm2; Path=/; Secure; HttpOnly;HttpOnly;Secure, GUEST_LANGUAGE_ID=en_US; Expires=Tue, 11-Nov-2025 14:31:26 GMT; Path=/; Secure;HttpOnly;Secure, COOKIE_SUPPORT=true; Expires=Tue, 11-Nov-2025 14:31:26 GMT; Path=/; Secure;HttpOnly;Secure, visid_incap_2225312=LR1cARm0Tb2h9t2an36wLjEVMmcAAAAAQUIPAAAAAAAdSzyW9OOwh/tgKvsggNkz; expires=Mon, 10 Nov 2025 23:10:41 GMT; HttpOnly; path=/; Domain=.athexgroup.gr; Secure; SameSite=None, nlbi_2225312=lfT5Uiu2pU1sr8KYVgaPnAAAAAD7N4wZ9vTtsycOxMEvrzkM; HttpOnly; path=/; Domain=.athexgroup.gr; Secure; SameSite=None, incap_ses_8219_2225312=eaBBHDCRkyy/1Uo/CcEPcj4VMmcA

In [24]:
s = "application/pdf"
pattern = r"^([a-z]+)/([a-z]+)"
m = re.match(pattern=pattern, string=s)
m.group(1)


'application'